In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

In [2]:
import torch
from dotenv import load_dotenv
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, GenerationConfig
import pandas as pd
import numpy as np
import random
import pickle
from IPython.display import display, Markdown
from copy import deepcopy
import seaborn as sns
import matplotlib.pyplot as plt

/home/pim/miniconda3/envs/ner/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### This package will be included in our repo, but it is only a prompt template

In [3]:
from golemai.nlp.prompts import QUERY_INTRO_NO_ANS, SYSTEM_MSG_RAG
from golemai.nlp.llm_module import prepare_prompt

In [4]:
load_dotenv(), torch.cuda.is_available()

(True, True)

In [5]:
DATA_DIR = 'data'
DATASET_FILE = 'test_gemma_resp_dola.parquet'
GEMMA_LABELS = 'gemma-2-9b-it-bnb-4bit.jsonl'
GEMMA_ATS_DIR = 'gemma_att_hidden_288_4110'
ATTENTIONS_DIR = 'attentions'
QUESTION_COL = 'question'
CONTEXT_COL = 'context'
ANSWER_COL = 'answer'

In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"
dataset_path = os.path.join("..", DATA_DIR, DATASET_FILE)
labels_path = os.path.join("..", DATA_DIR, GEMMA_LABELS)

In [7]:
df = pd.read_parquet(dataset_path)

In [8]:
df['dataset'].value_counts()

dataset
RAG-TGE        1417
HOTPOTQA-EN    1050
POLQA           710
HOTPOTQA-PL     645
REASON          288
Name: count, dtype: int64

In [9]:
df['context_length'] = df['context'].apply(lambda x: len(x.split()) if x else 0)

In [10]:
gemma_labels = pd.read_json(labels_path, lines=True)

In [11]:
gemma_labels['decision'].value_counts()

decision
True     3548
False     562
Name: count, dtype: int64

In [12]:
gemma_labels['decision'] = gemma_labels['decision'].astype(np.uint8)

In [13]:
examine_idxs = gemma_labels.iloc[288:377].loc[gemma_labels['decision'] == 0].index
gemma_labels.iloc[288:377].loc[gemma_labels['decision'] == 0]

,index,document,ground_truth,response,decision,gpt4_explanation,problematic_spans,cost,prompt
289,289,#Document#: `Dokument [1]:` Rower torowy – lek...,lonża,Nie mogę udzielić odpowiedzi na to pytanie na ...,0,The document provides information about variou...,"[""Nie mogę udzielić odpowiedzi na to pytanie n...",0.010320,You will be provided with a document and a pro...
292,292,"#Document#: `Dokument [ ""Chaenomeles"" ]:` Chae...",yes,"Yes, Mertensia is from the genus Boraginaceae ...",0,The document provides information about both M...,"[""Sasa is from the genus Bambusoideae""]",0.010615,You will be provided with a document and a pro...
295,295,"#Document#: `Dokument [ ""Plaza Kvinna"" ]:` Pla...",no,Yes. Both Nat Mags and Plaza Kvinna are publis...,0,"The document provides information about both ""...","[""Yes. Both Nat Mags and Plaza Kvinna are publ...",0.010750,You will be provided with a document and a pro...
300,300,"#Document#: `Dokument [ ""David Icke"" ]:` David...",1952,Nie mogę udzielić odpowiedzi na to pytanie na ...,0,The proposed answer is in Polish and translate...,"[""Nie mogę udzielić odpowiedzi na to pytanie n...",0.013015,You will be provided with a document and a pro...
312,312,"#Document#: `Dokument [ ""Houston Rockets"" ]:` ...","August 23, 1988","July 1, 1975",0,The document provides information about two in...,"[""July 1, 1975""]",0.010350,You will be provided with a document and a pro...
315,315,"#Document#: `Dokument [ ""That Green Gentleman ...","Psy 6 (Six Rules), Part 1","`Dokument [ ""Gentleman (Psy song)"" ]:` ""Gentle...",0,"The proposed answer states that ""Gentleman"" by...","[""Gentleman was released as a single, not on a...",0.011040,You will be provided with a document and a pro...
321,321,#Document#: `Dokument [1]:` Seria Mercedes-Ben...,Wersje coupé i kabriolet czwartej generacji sa...,Tak.,0,"The proposed answer ""Tak"" (Yes) is a response ...","[""Tak""]",0.011240,You will be provided with a document and a pro...
343,343,#Document#: `Dokument [1]:` Splice to kanadyjs...,Film Splice i producent filmowy Steve Hoban ma...,Kanadyjskiego,0,The proposed answer states that the nationalit...,"[""Kanadyjskiego""]",0.009095,You will be provided with a document and a pro...
346,346,#Document#: `Dokument [1]:` Lubię ten smutek –...,czubi,Nie mogę udzielić odpowiedzi na to pytanie na ...,0,"The document contains the phrase ""bajerujemy (...","[""Nie mogę udzielić odpowiedzi na to pytanie n...",0.007455,You will be provided with a document and a pro...
368,368,"#Document#: `Dokument [ ""Into the Wild (film)""...",Bill Pohlad,Tate Taylor directed and co-produced the 2014 ...,0,The document provides information about severa...,"[""Tate Taylor directed and co-produced""]",0.009475,You will be provided with a document and a pro...


In [14]:
df.iloc[examine_idxs]

,dataset,question_id,question,context,answer,gemma-2-9b-it-bnb-4bit,gemma-2-9b-it-bnb-4bit-dola,gemma-2-9b-it-bnb-4bit-few-shot-dola,gemma-2-9b-it-bnb-4bit-few-shot,context_length
289,POLQA,2759,Jak nazywa się taśma służąca do jeżdżenia konn...,`Dokument [1]:` Rower torowy – lekki rower na ...,lonża,Nie mogę udzielić odpowiedzi na to pytanie na ...,Nie mogę udzielić odpowiedzi na to pytanie na ...,Nie mogę udzielić odpowiedzi na to pytanie na ...,Nie mogę udzielić odpowiedzi na to pytanie na ...,544
292,HOTPOTQA-EN,5adf89625542995534e8c7dc,Are Mertensia and Sasa plants from differing g...,"`Dokument [ ""Chaenomeles"" ]:` Chaenomeles is a...",yes,"Yes, Mertensia is from the genus Boraginaceae ...",ANSWER: Yes. Mertensia belongs to the genus Bo...,"Tak, ponieważ Mertensia należy do rodzaju Bora...","Tak, Mertensia należy do rodzaju Boraginaceae,...",912
295,HOTPOTQA-EN,5ac2873d5542992f1f2b38ea,"Are the magazines, Nat Mags and Plaza Kvinna p...","`Dokument [ ""Plaza Kvinna"" ]:` Plaza Kvinna (m...",no,Yes. Both Nat Mags and Plaza Kvinna are publis...,Yes. Both Nat Mags and Plaza Kvinna are publi...,Nat Mags was published in Great Britain while ...,"Nat Mags was published in the United Kingdom, ...",912
300,HOTPOTQA-EN,5adce28f5542990d50227d52,Heinrick Palmgren hosts a show called Red Ice ...,"`Dokument [ ""David Icke"" ]:` David Vaughan Ick...",1952,Nie mogę udzielić odpowiedzi na to pytanie na ...,Nie mogę udzielić odpowiedzi na to pytanie na ...,Nie mogę udzielić odpowiedzi na to pytanie na ...,Nie mogę udzielić odpowiedzi na to pytanie na ...,1263
312,HOTPOTQA-EN,5ae294e35542996483e64a03,When was the Houston Rockets point guard recru...,"`Dokument [ ""Houston Rockets"" ]:` The Houston ...","August 23, 1988","July 1, 1975",Jeremy Lin was recruited by Frank Sullivan. Je...,Frank Sullivan recruited Jeremy Lin. Dokument ...,"Frank Sullivan recruited Jeremy Lin, who was b...",1222
315,HOTPOTQA-EN,5ab326d4554299194fa9357f,What album was the song Gentleman on?,"`Dokument [ ""That Green Gentleman (Things Have...","Psy 6 (Six Rules), Part 1","`Dokument [ ""Gentleman (Psy song)"" ]:` ""Gentle...","Dokument [ ""Gentleman (Psy song)"" ]:` ""Gentle...","Dokument [ ""Gentleman (Psy song)"" ]:` ""Gentlem...",The provided context mentions several songs ti...,925
321,RAG-TGE,3131,"Czy Mercedes-Benz Klasy E, wersje coupé i kab...",`Dokument [1]:` Seria Mercedes-Benz Klasy E (W...,Wersje coupé i kabriolet czwartej generacji sa...,Tak.,Tak.,Tak. Dokument [7]: Mercedes-Benz Klasa E (C207...,"Tak, Mercedes-Benz Klasy E, wersje coupé i kab...",708
343,RAG-TGE,2787,Jakiej narodowości/kraju pochodzenia dzielą f...,`Dokument [1]:` Splice to kanadyjsko-francuski...,Film Splice i producent filmowy Steve Hoban ma...,Kanadyjskiego,Kanadyjskiego,Dokument [1] mówi o filmie Splice jako kanadyj...,"Oba, film Splice i Steven Hoban, są kanadyjski...",442
346,POLQA,2654,"Proszę dokończyć przysłowie: „Kto się lubi, te...",`Dokument [1]:` Lubię ten smutek – album study...,czubi,Nie mogę udzielić odpowiedzi na to pytanie na ...,Nie mogę udzielić odpowiedzi na to pytanie na ...,Odpowiedź znajduje się w tekście: „Kto się czu...,"`Kto się lubi, ten się **czubi**`",341
368,HOTPOTQA-EN,5ac4190f5542997ea680ca09,Who directed and co-produced the 2014 American...,"`Dokument [ ""Into the Wild (film)"" ]:` Into th...",Bill Pohlad,Tate Taylor directed and co-produced the 2014 ...,Tate Taylor directed and co-produced the 2014 ...,Paul Edward Valentine Giamatti appeared in the...,Paul Edward Valentine Giamatti was in the film...,771


In [15]:
EXAMINE_IDX = 2916

In [ ]:
example_att_path = os.path.join(GEMMA_ATS_DIR, ATTENTIONS_DIR, f"{EXAMINE_IDX}.pkl")

with open(example_att_path, 'rb') as f:
    example_att = pickle.load(f)

In [21]:
hidden_path = os.path.join(GEMMA_ATS_DIR, 'hidden_states', f"{EXAMINE_IDX}.pkl")

with open(hidden_path, 'rb') as f:
    hidden = pickle.load(f)

In [25]:
hidden[0][0].shape, hidden[0][1].shape, hidden[1][0].shape

((471, 3584), (471, 3584), (3584,))

In [1]:
import torch

In [50]:
for i, att in enumerate(example_att):
    print(att[0].shape[-1] + i)

671
672
673
674
675
676
677


In [43]:
n_generated_tokens = len(example_att) - 1
_, n_prompt_tokens = example_att[0][0].shape[:2]

num_all_tokens = n_prompt_tokens + n_generated_tokens

res = np.concatenate([
    np.stack(tuple([t.squeeze() for t in token_att]), axis=0)[..., :num_all_tokens]
    if i == 0
    else np.stack(tuple([t.squeeze() for t in token_att]), axis=0)[..., :num_all_tokens][..., np.newaxis, :]
    for i, token_att in enumerate(example_att)
], axis=-2)

In [44]:
res.shape

(42, 16, 477, 477)

In [45]:
n_generated_tokens = len(hidden) - 1
_, n_prompt_tokens = hidden[0][0].shape[:2]

num_all_tokens = n_prompt_tokens + n_generated_tokens

res = np.concatenate([
    np.stack(tuple([t.squeeze() for t in token_att]), axis=0)[..., :num_all_tokens]
    if i == 0
    else np.stack(tuple([t.squeeze() for t in token_att]), axis=0)[:, np.newaxis, :]
    for i, token_att in enumerate(hidden)
], axis=-2)

In [46]:
res.shape

(43, 477, 3584)

In [20]:
res.shape

(42, 16, 477, 477)

In [17]:
len(example_att)

7

In [18]:
len(example_att[0])

42

In [40]:
%reload_ext autoreload
%autoreload 2

from attension_loader import AttentionLoader

In [41]:
att_loader = AttentionLoader(
    directory=os.path.join(GEMMA_ATS_DIR, ATTENTIONS_DIR),
    aggregation='max'
)

In [42]:
att_reduced = att_loader.get_reduced_attention()[0]

Processing 291.pkl
All attentions processed.


In [43]:
att_reduced.shape

(42, 16, 1251, 1252)

In [43]:
len(att_reduced), len(att_reduced[0]), att_reduced[0][0].shape, att_reduced[1][0].shape

(10, 42, (1242, 1242), (1243,))